<a href="https://colab.research.google.com/github/Sujoy-004/La-Liga-Score-Prediction/blob/main/predict_score_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load and inspect data


In [1]:
import pandas as pd

# Load the historical data
historical_data_url = "https://github.com/Sujoy-004/La-Liga-Score-Prediction/raw/main/matches_full.xlsx"
historical_data = pd.read_excel(historical_data_url)

# Load the future fixtures data
fixtures_data_url = "https://github.com/Sujoy-004/La-Liga-Score-Prediction/raw/main/la-liga-2025-UTC.xlsx"
fixtures_data = pd.read_excel(fixtures_data_url)

# Display the first 5 rows of historical_data
display(historical_data.head())

# Display the first 5 rows of fixtures_data
display(fixtures_data.head())

# Print concise summary of historical_data
historical_data.info()

# Print concise summary of fixtures_data
fixtures_data.info()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2,1,...,Match Report,NaN,17,5,18.6,1,1,1,2025,Barcelona
1,1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2,1,...,Match Report,NaN,13,5,16.6,0,0,0,2025,Barcelona
2,2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2,1,...,Match Report,NaN,22,5,19.3,1,0,0,2025,Barcelona
3,3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7,0,...,Match Report,NaN,23,11,13.7,1,0,0,2025,Barcelona
4,4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4,1,...,Match Report,NaN,20,9,19.1,0,0,0,2025,Barcelona


,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,2025-07-15 00:00:00,Estadio Municipal de Montilivi,Girona FC,Rayo Vallecano,NaN
1,2,1,2025-08-15 19:30:00,Estadio de la Cerámica,Villarreal CF,Real Oviedo,NaN
2,3,1,2025-08-16 15:00:00,Mendizorroza,Deportivo Alavés,Levante UD,NaN
3,4,1,2025-08-16 17:30:00,Estadi Mallorca Son Moix,RCD Mallorca,FC Barcelona,NaN
4,5,1,2025-08-16 19:30:00,Camp de Mestalla,Valencia CF,Real Sociedad,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4318 entries, 0 to 4317
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     4318 non-null   int64         
 1   date           4318 non-null   datetime64[ns]
 2   time           4318 non-null   object        
 3   comp           4318 non-null   object        
 4   round          4318 non-null   object        
 5   day            4318 non-null   object        
 6   venue          4318 non-null   object        
 7   result         4318 non-null   object        
 8   gf             4318 non-null   int64         
 9   ga             4318 non-null   int64         
 10  opponent       4318 non-null   object        
 11  xg             4318 non-null   float64       
 12  xga            4318 non-null   float64       
 13  poss           4318 non-null   int64         
 14  attendance     3344 non-null   float64       
 15  captain        4318 n

## Clean and preprocess data


In [2]:
# Create a function to clean column names
def clean_cols(df):
    """Cleans column names of a pandas DataFrame."""
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)
    return df

# Apply the cleaning function to both dataframes
historical_data = clean_cols(historical_data)
fixtures_data = clean_cols(fixtures_data)

# Convert date columns to datetime objects
historical_data['date'] = pd.to_datetime(historical_data['date'])
fixtures_data['date'] = pd.to_datetime(fixtures_data['date'])

# Identify unique team names in both dataframes
historical_teams = pd.concat([historical_data['team'], historical_data['opponent']]).unique()
fixtures_teams = pd.concat([fixtures_data['home_team'], fixtures_data['away_team']]).unique()

print("Unique teams in historical data:", historical_teams)
print("Unique teams in fixtures data:", fixtures_teams)

Unique teams in historical data: ['Barcelona' 'Atletico Madrid' 'Real Madrid' 'Athletic Club' 'Villarreal'
 'Real Betis' 'Rayo Vallecano' 'Mallorca' 'Real Sociedad' 'Celta Vigo'
 'Osasuna' 'Sevilla' 'Girona' 'Getafe' 'Espanyol' 'Leganes' 'Las Palmas'
 'Valencia' 'Alaves' 'Valladolid' 'Cadiz' 'Almeria' 'Granada' 'Elche'
 'Levante' 'Huesca' 'Eibar' 'Alavés' 'Betis' 'Leganés' 'Atlético Madrid'
 'Almería' 'Cádiz']
Unique teams in fixtures data: ['Girona FC' 'Villarreal CF' 'Deportivo Alavés' 'RCD Mallorca'
 'Valencia CF' 'Celta' 'Athletic Club' 'RCD Espanyol de Barcelona'
 'Elche CF' 'Real Madrid' 'Real Betis' 'Atlético de Madrid' 'Levante UD'
 'CA Osasuna' 'Real Sociedad' 'Real Oviedo' 'Sevilla FC' 'Rayo Vallecano'
 'FC Barcelona' 'Getafe CF']


In [3]:
# Manually create a mapping dictionary for inconsistent team names
team_mapping = {
    'Athletic Club': 'Athletic Club',
    'Atletico Madrid': 'Atletico Madrid',
    'Atlético Madrid': 'Atletico Madrid',
    'Barcelona': 'Barcelona',
    'FC Barcelona': 'Barcelona',
    'Real Madrid': 'Real Madrid',
    'Villarreal': 'Villarreal',
    'Villarreal CF': 'Villarreal',
    'Real Betis': 'Real Betis',
    'Betis': 'Real Betis',
    'Rayo Vallecano': 'Rayo Vallecano',
    'Mallorca': 'Mallorca',
    'RCD Mallorca': 'Mallorca',
    'Real Sociedad': 'Real Sociedad',
    'Celta Vigo': 'Celta Vigo',
    'Celta': 'Celta Vigo',
    'Osasuna': 'Osasuna',
    'CA Osasuna': 'Osasuna',
    'Sevilla': 'Sevilla',
    'Sevilla FC': 'Sevilla',
    'Girona': 'Girona',
    'Girona FC': 'Girona',
    'Getafe': 'Getafe',
    'Getafe CF': 'Getafe',
    'Espanyol': 'Espanyol',
    'RCD Espanyol de Barcelona': 'Espanyol',
    'Leganes': 'Leganes',
    'Leganés': 'Leganes',
    'Las Palmas': 'Las Palmas',
    'Valencia': 'Valencia',
    'Valencia CF': 'Valencia',
    'Alaves': 'Alaves',
    'Alavés': 'Alaves',
    'Deportivo Alavés': 'Alaves',
    'Valladolid': 'Valladolid',
    'Cadiz': 'Cadiz',
    'Cádiz': 'Cadiz',
    'Almeria': 'Almeria',
    'Almería': 'Almeria',
    'Granada': 'Granada',
    'Elche': 'Elche',
    'Elche CF': 'Elche',
    'Levante': 'Levante',
    'Levante UD': 'Levante',
    'Huesca': 'Huesca',
    'Eibar': 'Eibar',
    'Real Oviedo': 'Real Oviedo'
}

# Apply the mapping to historical_data
historical_data['team'] = historical_data['team'].map(team_mapping).fillna(historical_data['team'])
historical_data['opponent'] = historical_data['opponent'].map(team_mapping).fillna(historical_data['opponent'])

# Apply the mapping and rename columns in fixtures_data
fixtures_data['home_team'] = fixtures_data['home_team'].map(team_mapping).fillna(fixtures_data['home_team'])
fixtures_data['away_team'] = fixtures_data['away_team'].map(team_mapping).fillna(fixtures_data['away_team'])

# Verify the changes
display(historical_data.head())
display(fixtures_data.head())

,unnamed_0,date,time,comp,round,day,venue,result,gf,ga,...,match_report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2,1,...,Match Report,NaN,17,5,18.6,1,1,1,2025,Barcelona
1,1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2,1,...,Match Report,NaN,13,5,16.6,0,0,0,2025,Barcelona
2,2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2,1,...,Match Report,NaN,22,5,19.3,1,0,0,2025,Barcelona
3,3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7,0,...,Match Report,NaN,23,11,13.7,1,0,0,2025,Barcelona
4,4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4,1,...,Match Report,NaN,20,9,19.1,0,0,0,2025,Barcelona


,match_number,round_number,date,location,home_team,away_team,result
0,1,1,2025-07-15 00:00:00,Estadio Municipal de Montilivi,Girona,Rayo Vallecano,NaN
1,2,1,2025-08-15 19:30:00,Estadio de la Cerámica,Villarreal,Real Oviedo,NaN
2,3,1,2025-08-16 15:00:00,Mendizorroza,Alaves,Levante,NaN
3,4,1,2025-08-16 17:30:00,Estadi Mallorca Son Moix,Mallorca,Barcelona,NaN
4,5,1,2025-08-16 19:30:00,Camp de Mestalla,Valencia,Real Sociedad,NaN


## Feature engineering


In [4]:
# Filter historical_data to include matches from the 2020 season onwards
matches_df = historical_data[historical_data['season'] >= 2020].copy()

# Calculate goals scored and conceded per match for each team in each season
matches_df['home_goals_per_match'] = matches_df.groupby(['season', 'team'])['gf'].transform(lambda x: x.expanding().mean().shift(1)).fillna(0)
matches_df['home_conceded_per_match'] = matches_df.groupby(['season', 'team'])['ga'].transform(lambda x: x.expanding().mean().shift(1)).fillna(0)

# For away matches, the 'team' column represents the away team, and 'opponent' represents the home team.
# We need to calculate stats for the away team based on their away games.
# First, filter for away games and calculate stats based on the 'team' and 'opponent' columns.
away_matches = matches_df[matches_df['venue'] == 'Away'].copy()
away_matches['away_goals_per_match'] = away_matches.groupby(['season', 'team'])['gf'].transform(lambda x: x.expanding().mean().shift(1)).fillna(0)
away_matches['away_conceded_per_match'] = away_matches.groupby(['season', 'team'])['ga'].transform(lambda x: x.expanding().mean().shift(1)).fillna(0)

# Merge the calculated away stats back to the main matches_df.
# Need to merge on season, date, and team (which is the away team in the away_matches df)
matches_df = matches_df.merge(
    away_matches[['season', 'date', 'team', 'away_goals_per_match', 'away_conceded_per_match']],
    on=['season', 'date', 'team'],
    how='left',
    suffixes=('', '_away')
)

# Need to handle matches where the team is the home team.
# In these cases, we need the away team's stats, which are calculated from their away games.
# The away team in the main matches_df is the 'opponent' when the 'venue' is 'Home'.
home_matches = matches_df[matches_df['venue'] == 'Home'].copy()
home_matches = home_matches.merge(
    away_matches[['season', 'date', 'team', 'away_goals_per_match', 'away_conceded_per_match']],
    left_on=['season', 'date', 'opponent'],
    right_on=['season', 'date', 'team'],
    how='left',
    suffixes=('', '_away_opponent')
)

# Update the main matches_df with the away team's stats for home matches
matches_df.loc[matches_df['venue'] == 'Home', 'away_goals_per_match'] = home_matches['away_goals_per_match_away_opponent'].values
matches_df.loc[matches_df['venue'] == 'Home', 'away_conceded_per_match'] = home_matches['away_conceded_per_match_away_opponent'].values

# Fill remaining NaNs in away stats (for the very first away game of a team in a season)
matches_df['away_goals_per_match'] = matches_df['away_goals_per_match'].fillna(0)
matches_df['away_conceded_per_match'] = matches_df['away_conceded_per_match'].fillna(0)


# Calculate home and away win rates
def calculate_win_rate(df, venue_type):
    df['wins'] = (df['result'] == 'W').astype(int)
    df['draws'] = (df['result'] == 'D').astype(int)
    df['losses'] = (df['result'] == 'L').astype(int)
    df[f'{venue_type}_matches_played'] = df.groupby(['season', 'team']).cumcount()
    df[f'{venue_type}_wins_cumulative'] = df.groupby(['season', 'team'])['wins'].cumsum()
    df[f'{venue_type}_draws_cumulative'] = df.groupby(['season', 'team'])['draws'].cumsum()
    df[f'{venue_type}_losses_cumulative'] = df.groupby(['season', 'team'])['losses'].cumsum()

    # Calculate win rate based on matches played up to the previous game
    df[f'{venue_type}_win_rate'] = (df[f'{venue_type}_wins_cumulative'].shift(1).fillna(0) / df[f'{venue_type}_matches_played'].shift(1).fillna(0)).fillna(0)

    # Drop cumulative columns
    df = df.drop(columns=[f'{venue_type}_wins_cumulative', f'{venue_type}_draws_cumulative', f'{venue_type}_losses_cumulative', f'{venue_type}_matches_played', 'wins', 'draws', 'losses'])
    return df

# Calculate home win rate
home_matches_df = matches_df[matches_df['venue'] == 'Home'].copy()
home_matches_df = calculate_win_rate(home_matches_df, 'home')

# Calculate away win rate
away_matches_df = matches_df[matches_df['venue'] == 'Away'].copy()
away_matches_df = calculate_win_rate(away_matches_df, 'away')

# Merge win rates back to matches_df
matches_df = matches_df.merge(home_matches_df[['season', 'date', 'team', 'home_win_rate']], on=['season', 'date', 'team'], how='left')
matches_df = matches_df.merge(away_matches_df[['season', 'date', 'team', 'away_win_rate']], on=['season', 'date', 'team'], how='left')


# Need to get the opponent's win rate
# When venue is Home, we need the opponent's away_win_rate
matches_df.loc[matches_df['venue'] == 'Home', 'away_win_rate'] = matches_df.loc[matches_df['venue'] == 'Home'].merge(
    away_matches_df[['season', 'date', 'team', 'away_win_rate']],
    left_on=['season', 'date', 'opponent'],
    right_on=['season', 'date', 'team'],
    how='left'
)['away_win_rate_y'].values

# When venue is Away, we need the opponent's home_win_rate
matches_df.loc[matches_df['venue'] == 'Away', 'home_win_rate'] = matches_df.loc[matches_df['venue'] == 'Away'].merge(
    home_matches_df[['season', 'date', 'team', 'home_win_rate']],
    left_on=['season', 'date', 'opponent'],
    right_on=['season', 'date', 'team'],
    how='left'
)['home_win_rate_y'].values

# Fill remaining NaNs in win rates (for the very first game of a team in a season)
matches_df['home_win_rate'] = matches_df['home_win_rate'].fillna(0)
matches_df['away_win_rate'] = matches_df['away_win_rate'].fillna(0)


# Create the target variable based on the match result from the perspective of the home team
matches_df['target'] = matches_df['result'].apply(lambda x: 'W' if x == 'W' else ('D' if x == 'D' else 'L'))

display(matches_df.head())

,unnamed_0,date,time,comp,round,day,venue,result,gf,ga,...,pkatt,season,team,home_goals_per_match,home_conceded_per_match,away_goals_per_match,away_conceded_per_match,home_win_rate,away_win_rate,target
0,0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2,1,...,1,2025,Barcelona,0.00,0.00,0.0,0.0,0.25,0.000000,W
1,1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2,1,...,0,2025,Barcelona,2.00,1.00,0.0,0.0,0.00,0.461538,W
2,2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2,1,...,0,2025,Barcelona,2.00,1.00,2.0,1.0,0.50,inf,W
3,3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7,0,...,0,2025,Barcelona,2.00,1.00,0.0,3.0,inf,0.000000,W
4,4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4,1,...,0,2025,Barcelona,3.25,0.75,2.0,1.0,inf,2.000000,W


In [5]:
# Calculate recent form features (last 5 matches)
def calculate_recent_form(df, team_col, venue_type):
    df['points'] = df['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))

    # Calculate outcomes for the team
    df['is_win'] = (df['result'] == 'W').astype(int)
    df['is_draw'] = (df['result'] == 'D').astype(int)
    df['is_loss'] = (df['result'] == 'L').astype(int)

    # Calculate rolling sum of outcomes and points
    df[f'{venue_type}_form_wins'] = df.groupby(['season', team_col])['is_win'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).sum().fillna(0))
    df[f'{venue_type}_form_draws'] = df.groupby(['season', team_col])['is_draw'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).sum().fillna(0))
    df[f'{venue_type}_form_losses'] = df.groupby(['season', team_col])['is_loss'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).sum().fillna(0))
    df[f'{venue_type}_form_points'] = df.groupby(['season', team_col])['points'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).sum().fillna(0))

    # Calculate rolling sum of goals for and against
    df[f'{venue_type}_form_gf'] = df.groupby(['season', team_col])['gf'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).sum().fillna(0))
    df[f'{venue_type}_form_ga'] = df.groupby(['season', team_col])['ga'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).sum().fillna(0))

    # Drop temporary columns
    df = df.drop(columns=['is_win', 'is_draw', 'is_loss', 'points'])
    return df

# Calculate home team recent form
home_matches_form = matches_df[matches_df['venue'] == 'Home'].copy()
home_matches_form = calculate_recent_form(home_matches_form, 'team', 'home')

# Calculate away team recent form
away_matches_form = matches_df[matches_df['venue'] == 'Away'].copy()
away_matches_form = calculate_recent_form(away_matches_form, 'team', 'away')

# Merge recent form features back to matches_df
matches_df = matches_df.merge(home_matches_form[['season', 'date', 'team', 'home_form_wins', 'home_form_draws', 'home_form_losses', 'home_form_points', 'home_form_gf', 'home_form_ga']], on=['season', 'date', 'team'], how='left')
matches_df = matches_df.merge(away_matches_form[['season', 'date', 'team', 'away_form_wins', 'away_form_draws', 'away_form_losses', 'away_form_points', 'away_form_gf', 'away_form_ga']], on=['season', 'date', 'team'], how='left')

# Need to get the opponent's recent form
# When venue is Home, we need the opponent's away_form
matches_df.loc[matches_df['venue'] == 'Home', ['away_form_wins', 'away_form_draws', 'away_form_losses', 'away_form_points', 'away_form_gf', 'away_form_ga']] = matches_df.loc[matches_df['venue'] == 'Home'].merge(
    away_matches_form[['season', 'date', 'team', 'away_form_wins', 'away_form_draws', 'away_form_losses', 'away_form_points', 'away_form_gf', 'away_form_ga']],
    left_on=['season', 'date', 'opponent'],
    right_on=['season', 'date', 'team'],
    how='left'
)[['away_form_wins_y', 'away_form_draws_y', 'away_form_losses_y', 'away_form_points_y', 'away_form_gf_y', 'away_form_ga_y']].values

# When venue is Away, we need the opponent's home_form
matches_df.loc[matches_df['venue'] == 'Away', ['home_form_wins', 'home_form_draws', 'home_form_losses', 'home_form_points', 'home_form_gf', 'home_form_ga']] = matches_df.loc[matches_df['venue'] == 'Away'].merge(
    home_matches_form[['season', 'date', 'team', 'home_form_wins', 'home_form_draws', 'home_form_losses', 'home_form_points', 'home_form_gf', 'home_form_ga']],
    left_on=['season', 'date', 'opponent'],
    right_on=['season', 'date', 'team'],
    how='left'
)[['home_form_wins_y', 'home_form_draws_y', 'home_form_losses_y', 'home_form_points_y', 'home_form_gf_y', 'home_form_ga_y']].values

# Fill initial NaN values for recent form (first few matches) with 0
form_cols = [f'{venue}_form_{stat}' for venue in ['home', 'away'] for stat in ['wins', 'draws', 'losses', 'points', 'gf', 'ga']]
matches_df[form_cols] = matches_df[form_cols].fillna(0)


# Calculate head-to-head records
def calculate_h2h(df):
    df['h2h_home_wins'] = 0
    df['h2h_draws'] = 0
    df['h2h_away_wins'] = 0

    for index, row in df.iterrows():
        home_team = row['team']
        away_team = row['opponent']
        match_date = row['date']
        season = row['season']

        # Filter historical matches between these two teams before the current match date and within the same season or previous seasons
        h2h_matches = matches_df[(matches_df['date'] < match_date) & (
            ((matches_df['team'] == home_team) & (matches_df['opponent'] == away_team)) |
            ((matches_df['team'] == away_team) & (matches_df['opponent'] == home_team))
        )].copy()

        # Calculate h2h stats from the perspective of the current home team
        home_wins = h2h_matches[(h2h_matches['team'] == home_team) & (h2h_matches['result'] == 'W')].shape[0] + \
                    h2h_matches[(h2h_matches['opponent'] == home_team) & (h2h_matches['result'] == 'L')].shape[0]
        draws = h2h_matches[h2h_matches['result'] == 'D'].shape[0]
        away_wins = h2h_matches[(h2h_matches['team'] == away_team) & (h2h_matches['result'] == 'W')].shape[0] + \
                    h2h_matches[(h2h_matches['opponent'] == away_team) & (h2h_matches['result'] == 'L')].shape[0]

        df.loc[index, 'h2h_home_wins'] = home_wins
        df.loc[index, 'h2h_draws'] = draws
        df.loc[index, 'h2h_away_wins'] = away_wins

    return df

# Calculate head-to-head records for home matches
home_matches_h2h = matches_df[matches_df['venue'] == 'Home'].copy()
home_matches_h2h = calculate_h2h(home_matches_h2h)

# Merge h2h features back to matches_df for home matches
matches_df = matches_df.merge(home_matches_h2h[['season', 'date', 'team', 'opponent', 'h2h_home_wins', 'h2h_draws', 'h2h_away_wins']], on=['season', 'date', 'team', 'opponent'], how='left')

# For away matches, the 'team' is the away team and 'opponent' is the home team.
# We need to calculate h2h from the perspective of the actual home team ('opponent' column).
away_matches_h2h = matches_df[matches_df['venue'] == 'Away'].copy()
away_matches_h2h = calculate_h2h(away_matches_h2h)

# Merge h2h features back to matches_df for away matches, aligning by the actual home and away teams
matches_df = matches_df.merge(
    away_matches_h2h[['season', 'date', 'team', 'opponent', 'h2h_home_wins', 'h2h_draws', 'h2h_away_wins']],
    left_on=['season', 'date', 'team', 'opponent'],
    right_on=['season', 'date', 'opponent', 'team'], # Note the swapped columns for merging
    how='left',
    suffixes=('', '_away_h2h')
)

# Update the h2h columns for away matches with the values calculated from the home team's perspective
matches_df.loc[matches_df['venue'] == 'Away', 'h2h_home_wins'] = matches_df.loc[matches_df['venue'] == 'Away', 'h2h_away_wins_away_h2h']
matches_df.loc[matches_df['venue'] == 'Away', 'h2h_draws'] = matches_df.loc[matches_df['venue'] == 'Away', 'h2h_draws_away_h2h']
matches_df.loc[matches_df['venue'] == 'Away', 'h2h_away_wins'] = matches_df.loc[matches_df['venue'] == 'Away', 'h2h_home_wins_away_h2h']

# Drop the temporary merge columns
matches_df = matches_df.drop(columns=['h2h_home_wins_away_h2h', 'h2h_draws_away_h2h', 'h2h_away_wins_away_h2h'])


# Filter matches_df to only include rows where venue is 'Home'
matches_df = matches_df[matches_df['venue'] == 'Home'].copy()

# Fill any remaining NaN values with 0 (should mostly be for the very first matches)
matches_df.fillna(0, inplace=True)


# Display the first few rows with the new features
display(matches_df.head())

,unnamed_0,date,time,comp,round,day,venue,result,gf,ga,...,away_form_draws,away_form_losses,away_form_points,away_form_gf,away_form_ga,h2h_home_wins,h2h_draws,h2h_away_wins,team_away_h2h,opponent_away_h2h
1,1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2,1,...,0.0,0.0,0.0,0.0,0.0,14.0,4.0,2.0,Athletic Club,Barcelona
3,3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7,0,...,0.0,1.0,0.0,0.0,3.0,10.0,0.0,2.0,Valladolid,Barcelona
6,7,2024-09-25,21:00,La Liga,Matchweek 7,Wed,Home,W,1,0,...,1.0,2.0,1.0,2.0,4.0,12.0,6.0,2.0,Getafe,Barcelona
9,11,2024-10-20,21:00,La Liga,Matchweek 10,Sun,Home,W,5,1,...,3.0,1.0,3.0,4.0,5.0,14.0,6.0,0.0,Sevilla,Barcelona
11,14,2024-11-03,16:15,La Liga,Matchweek 12,Sun,Home,W,3,1,...,1.0,4.0,1.0,2.0,10.0,6.0,6.0,0.0,Espanyol,Barcelona


In [6]:
# Select the features to be used for the ML model, using 'team' and 'opponent' initially
features_df = matches_df[['date', 'team', 'opponent', 'target', 'season',
                          'home_goals_per_match', 'home_conceded_per_match',
                          'away_goals_per_match', 'away_conceded_per_match',
                          'home_win_rate', 'away_win_rate',
                          'home_form_wins', 'home_form_draws', 'home_form_losses',
                          'home_form_points', 'home_form_gf', 'home_form_ga',
                          'away_form_wins', 'away_form_draws', 'away_form_losses',
                          'away_form_points', 'away_form_gf', 'away_form_ga',
                          'h2h_home_wins', 'h2h_draws', 'h2h_away_wins']].copy()

# Rename 'team' to 'home_team' and 'opponent' to 'away_team' for clarity
features_df = features_df.rename(columns={'team': 'home_team', 'opponent': 'away_team'})

# Display the first few rows of the features_df
display(features_df.head())

,date,home_team,away_team,target,season,home_goals_per_match,home_conceded_per_match,away_goals_per_match,away_conceded_per_match,home_win_rate,...,home_form_ga,away_form_wins,away_form_draws,away_form_losses,away_form_points,away_form_gf,away_form_ga,h2h_home_wins,h2h_draws,h2h_away_wins
1,2024-08-24,Barcelona,Athletic Club,W,2025,2.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,4.0,2.0
3,2024-08-31,Barcelona,Valladolid,W,2025,2.000000,1.000000,0.000000,3.000000,inf,...,1.0,0.0,0.0,1.0,0.0,0.0,3.0,10.0,0.0,2.0
6,2024-09-25,Barcelona,Getafe,W,2025,3.666667,0.833333,0.666667,1.333333,2.000000,...,1.0,0.0,1.0,2.0,1.0,2.0,4.0,12.0,6.0,2.0
9,2024-10-20,Barcelona,Sevilla,W,2025,3.111111,1.000000,1.000000,1.250000,1.500000,...,1.0,0.0,3.0,1.0,3.0,4.0,5.0,14.0,6.0,0.0
11,2024-11-03,Barcelona,Espanyol,W,2025,3.363636,0.909091,0.400000,2.000000,1.333333,...,2.0,0.0,1.0,4.0,1.0,2.0,10.0,6.0,6.0,0.0


## Target encoding

In [7]:
# Create the mapping dictionary
target_mapping = {'W': 2, 'D': 1, 'L': 0}

# Apply the mapping to create the encoded target column
features_df['target_encoded'] = features_df['target'].map(target_mapping)

# Display the first few rows to show the new column
display(features_df.head())

,date,home_team,away_team,target,season,home_goals_per_match,home_conceded_per_match,away_goals_per_match,away_conceded_per_match,home_win_rate,...,away_form_wins,away_form_draws,away_form_losses,away_form_points,away_form_gf,away_form_ga,h2h_home_wins,h2h_draws,h2h_away_wins,target_encoded
1,2024-08-24,Barcelona,Athletic Club,W,2025,2.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,4.0,2.0,2
3,2024-08-31,Barcelona,Valladolid,W,2025,2.000000,1.000000,0.000000,3.000000,inf,...,0.0,0.0,1.0,0.0,0.0,3.0,10.0,0.0,2.0,2
6,2024-09-25,Barcelona,Getafe,W,2025,3.666667,0.833333,0.666667,1.333333,2.000000,...,0.0,1.0,2.0,1.0,2.0,4.0,12.0,6.0,2.0,2
9,2024-10-20,Barcelona,Sevilla,W,2025,3.111111,1.000000,1.000000,1.250000,1.500000,...,0.0,3.0,1.0,3.0,4.0,5.0,14.0,6.0,0.0,2
11,2024-11-03,Barcelona,Espanyol,W,2025,3.363636,0.909091,0.400000,2.000000,1.333333,...,0.0,1.0,4.0,1.0,2.0,10.0,6.0,6.0,0.0,2


## Data splitting

In [8]:
# Define masks for splitting based on season
train_mask = features_df['season'] < 2024
val_mask = features_df['season'] == 2024
test_mask = features_df['season'] == 2025

# Define the features to use for the model (excluding 'date', 'home_team', 'away_team', 'target', 'target_encoded')
model_features = features_df.columns.drop(['date', 'home_team', 'away_team', 'target', 'target_encoded'])

# Create the training set
X_train = features_df.loc[train_mask, model_features]
y_train_encoded = features_df.loc[train_mask, 'target_encoded']

# Create the validation set
X_val = features_df.loc[val_mask, model_features]
y_val_encoded = features_df.loc[val_mask, 'target_encoded']

# Create the test set
X_test = features_df.loc[test_mask, model_features]
y_test_encoded = features_df.loc[test_mask, 'target_encoded']

# Display the shapes of the resulting datasets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train_encoded:", y_train_encoded.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val_encoded:", y_val_encoded.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test_encoded:", y_test_encoded.shape)

Shape of X_train: (1520, 22)
Shape of y_train_encoded: (1520,)
Shape of X_val: (380, 22)
Shape of y_val_encoded: (380,)
Shape of X_test: (259, 22)
Shape of y_test_encoded: (259,)


## Model selection and training

In [9]:
import numpy as np

# Check for infinite values in X_train
print("Checking for infinite values in X_train:")
display(X_train.replace([np.inf, -np.inf], np.nan).isnull().sum()[X_train.replace([np.inf, -np.inf], np.nan).isnull().sum() > 0])

# Replace infinite values with NaN and then fill NaN with 0
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
X_val = X_val.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

# Verify that infinite values are handled
print("\nChecking for infinite values in X_train after handling:")
display(X_train.replace([np.inf, -np.inf], np.nan).isnull().sum()[X_train.replace([np.inf, -np.inf], np.nan).isnull().sum() > 0])

# Now, try training the models again
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

# Instantiate each model with random_state=42 for reproducibility.
rf_model = RandomForestClassifier(random_state=42)
xgb_model = XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss', random_state=42)
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

# Train each model using their respective .fit() methods
rf_model.fit(X_train, y_train_encoded)
xgb_model.fit(X_train, y_train_encoded)
lgb_model.fit(X_train, y_train_encoded)

# Store the trained models in a dictionary
models = {
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'LightGBM': lgb_model
}

print("\nModels trained successfully:")
for name in models:
    print(f"- {name}")

Checking for infinite values in X_train:


,0
home_win_rate,29
away_win_rate,24



Checking for infinite values in X_train after handling:


,0


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1116
[LightGBM] [Info] Number of data points in the train set: 1520, number of used features: 22
[LightGBM] [Info] Start training from score -1.269682
[LightGBM] [Info] Start training from score -1.300600
[LightGBM] [Info] Start training from score -0.805844

Models trained successfully:
- Random Forest
- XGBoost
- LightGBM


## Model evaluation

In [10]:
from sklearn.metrics import accuracy_score

# Dictionary to store validation accuracies
val_accuracies = {}

# Iterate through the trained models
for name, model in models.items():
    # Make predictions on the validation set
    y_pred_val = model.predict(X_val)

    # Calculate accuracy
    accuracy = accuracy_score(y_val_encoded, y_pred_val)

    # Store the accuracy
    val_accuracies[name] = accuracy

    # Print the validation accuracy
    print(f"{name} Validation Accuracy: {accuracy:.4f}")

Random Forest Validation Accuracy: 0.5053
XGBoost Validation Accuracy: 0.4474
LightGBM Validation Accuracy: 0.4447


## Implement Prediction on Fixture Data


In [12]:
# Initialize lists to store features and predictions for each fixture
fixture_features_list = []

# Iterate through each row in the fixtures_data dataframe
for index, fixture_row in fixtures_data.iterrows():
    fixture_date = fixture_row['date']
    home_team = fixture_row['home_team']
    away_team = fixture_row['away_team']
    fixture_season = fixture_date.year # Assuming season is the year of the match

    # Use historical data from previous seasons and matches within the current season up to the fixture date
    historical_subset = matches_df[(matches_df['date'] < fixture_date) & (matches_df['season'] <= fixture_season)].copy()

    # Calculate home team's average goals scored and conceded per match (from their home games in historical_subset)
    home_team_historical_home = historical_subset[(historical_subset['team'] == home_team) & (historical_subset['venue'] == 'Home')].copy()
    home_goals_per_match = home_team_historical_home['gf'].mean() if not home_team_historical_home.empty else 0
    home_conceded_per_match = home_team_historical_home['ga'].mean() if not home_team_historical_home.empty else 0

    # Calculate away team's average goals scored and conceded per match (from their away games in historical_subset)
    away_team_historical_away = historical_subset[(historical_subset['team'] == away_team) & (historical_subset['venue'] == 'Away')].copy()
    away_goals_per_match = away_team_historical_away['gf'].mean() if not away_team_historical_away.empty else 0
    away_conceded_per_match = away_team_historical_away['ga'].mean() if not away_team_historical_away.empty else 0

    # Calculate home and away win rates
    home_team_historical = historical_subset[historical_subset['team'] == home_team].copy()
    home_wins = home_team_historical[home_team_historical['result'] == 'W'].shape[0]
    home_matches_played = home_team_historical.shape[0]
    home_win_rate = home_wins / home_matches_played if home_matches_played > 0 else 0

    away_team_historical = historical_subset[historical_subset['team'] == away_team].copy()
    away_wins = away_team_historical[away_team_historical['result'] == 'W'].shape[0]
    away_matches_played = away_team_historical.shape[0]
    away_win_rate = away_wins / away_matches_played if away_matches_played > 0 else 0

    # Calculate recent form (last 5 matches) - need to consider matches across venues
    home_team_recent_form = historical_subset[(historical_subset['team'] == home_team)].tail(5).copy()
    home_form_wins = (home_team_recent_form['result'] == 'W').sum()
    home_form_draws = (home_team_recent_form['result'] == 'D').sum()
    home_form_losses = (home_team_recent_form['result'] == 'L').sum()
    home_form_points = (home_team_recent_form['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))).sum()
    home_form_gf = home_team_recent_form['gf'].sum()
    home_form_ga = home_team_recent_form['ga'].sum()

    away_team_recent_form = historical_subset[(historical_subset['team'] == away_team)].tail(5).copy()
    away_form_wins = (away_team_recent_form['result'] == 'W').sum()
    away_form_draws = (away_team_recent_form['result'] == 'D').sum()
    away_form_losses = (away_team_recent_form['result'] == 'L').sum()
    away_form_points = (away_team_recent_form['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))).sum()
    away_form_gf = away_team_recent_form['gf'].sum()
    away_form_ga = away_team_recent_form['ga'].sum()

    # Calculate head-to-head records
    h2h_matches = historical_subset[
        ((historical_subset['team'] == home_team) & (historical_subset['opponent'] == away_team)) |
        ((historical_subset['team'] == away_team) & (historical_subset['opponent'] == home_team))
    ].copy()

    h2h_home_wins = h2h_matches[(h2h_matches['team'] == home_team) & (h2h_matches['result'] == 'W')].shape[0] + \
                    h2h_matches[(h2h_matches['opponent'] == home_team) & (h2h_matches['result'] == 'L')].shape[0]
    h2h_draws = h2h_matches[h2h_matches['result'] == 'D'].shape[0]
    h2h_away_wins = h2h_matches[(h2h_matches['team'] == away_team) & (h2h_matches['result'] == 'W')].shape[0] + \
                    h2h_matches[(h2h_matches['opponent'] == away_team) & (h2h_matches['result'] == 'L')].shape[0]


    # Create a dictionary of features for the current fixture
    fixture_features = {
        'season': fixture_season,
        'home_goals_per_match': home_goals_per_match,
        'home_conceded_per_match': home_conceded_per_match,
        'away_goals_per_match': away_goals_per_match,
        'away_conceded_per_match': away_conceded_per_match,
        'home_win_rate': home_win_rate,
        'away_win_rate': away_win_rate,
        'home_form_wins': home_form_wins,
        'home_form_draws': home_form_draws,
        'home_form_losses': home_form_losses,
        'home_form_points': home_form_points,
        'home_form_gf': home_form_gf,
        'home_form_ga': home_form_ga,
        'away_form_wins': away_form_wins,
        'away_form_draws': away_form_draws,
        'away_form_losses': away_form_losses,
        'away_form_points': away_form_points,
        'away_form_gf': away_form_gf,
        'away_form_ga': away_form_ga,
        'h2h_home_wins': h2h_home_wins,
        'h2h_draws': h2h_draws,
        'h2h_away_wins': h2h_away_wins
    }

    fixture_features_list.append(fixture_features)

# Convert the list of dictionaries to a DataFrame
fixture_features_df = pd.DataFrame(fixture_features_list)

# Reindex to match the order of columns in X_train.columns
fixture_features_df = fixture_features_df.reindex(columns=X_train.columns, fill_value=0)

# Handle infinite values and fill NaNs in the created features for fixtures
fixture_features_df = fixture_features_df.replace([np.inf, -np.inf], np.nan).fillna(0)

# Select the best performing model based on validation accuracy
best_model_name = max(val_accuracies, key=val_accuracies.get)
best_model = models[best_model_name]

# Define the reverse mapping for encoded predictions
reverse_target_mapping = {v: k for k, v in target_mapping.items()}

# Use the best model to make predictions on the fixture features
fixture_predictions_encoded = best_model.predict(fixture_features_df)

# Convert encoded predictions back to labels
fixture_predictions = pd.Series(fixture_predictions_encoded).map(reverse_target_mapping)

# Add the predictions to the fixtures_data dataframe
fixtures_data['predicted_result'] = fixture_predictions.values

# Display the fixtures_data with the predicted results
print("2025-26 Fixtures with Predicted Results:")
display(fixtures_data.head())

print("'W' means the Home Team is predicted to win.")
print("'D' means a Draw is predicted.")
print("'L' means the Home Team is predicted to lose (which means the Away Team is predicted to win).")

2025-26 Fixtures with Predicted Results:


,match_number,round_number,date,location,home_team,away_team,result,predicted_result
0,1,1,2025-07-15 00:00:00,Estadio Municipal de Montilivi,Girona,Rayo Vallecano,NaN,W
1,2,1,2025-08-15 19:30:00,Estadio de la Cerámica,Villarreal,Real Oviedo,NaN,W
2,3,1,2025-08-16 15:00:00,Mendizorroza,Alaves,Levante,NaN,D
3,4,1,2025-08-16 17:30:00,Estadi Mallorca Son Moix,Mallorca,Barcelona,NaN,L
4,5,1,2025-08-16 19:30:00,Camp de Mestalla,Valencia,Real Sociedad,NaN,D


'W' means the Home Team is predicted to win.
'D' means a Draw is predicted.
'L' means the Home Team is predicted to lose (which means the Away Team is predicted to win).


## Predict Specific Head-to-Head Matches

In [13]:
# Display the unique standardized team names
print("Standardized Team Names:")
display(historical_data['team'].unique())

Standardized Team Names:


array(['Barcelona', 'Atletico Madrid', 'Real Madrid', 'Athletic Club',
       'Villarreal', 'Real Betis', 'Rayo Vallecano', 'Mallorca',
       'Real Sociedad', 'Celta Vigo', 'Osasuna', 'Sevilla', 'Girona',
       'Getafe', 'Espanyol', 'Leganes', 'Las Palmas', 'Valencia',
       'Alaves', 'Valladolid', 'Cadiz', 'Almeria', 'Granada', 'Elche',
       'Levante', 'Huesca', 'Eibar'], dtype=object)

In [14]:
# Get team names from user input
input_home_team = input("Enter the name of the Home Team: ")
input_away_team = input("Enter the name of the Away Team: ")

# Filter fixtures_data for matches between the two input teams in the 2025-26 season
team1 = team_mapping.get(input_home_team, input_home_team)
team2 = team_mapping.get(input_away_team, input_away_team)


specific_fixtures = fixtures_data[
    ((fixtures_data['home_team'] == team1) & (fixtures_data['away_team'] == team2)) |
    ((fixtures_data['home_team'] == team2) & (fixtures_data['away_team'] == team1))
].copy()

if specific_fixtures.empty:
    print(f"No matches found between {input_home_team} and {input_away_team} in the 2025-26 fixtures.")
else:
    print(f"\nPredicting matches between {team1} and {team2} in the 2025-26 season:")

    # Initialize list to store features for specific fixtures
    specific_fixture_features_list = []

    # Iterate through each specific fixture to create features
    for index, fixture_row in specific_fixtures.iterrows():
        fixture_date = fixture_row['date']
        home_team = fixture_row['home_team']
        away_team = fixture_row['away_team']
        fixture_season = fixture_date.year

        # Filter historical data up to the fixture date and including the fixture season
        historical_subset = matches_df[(matches_df['date'] < fixture_date) & (matches_df['season'] <= fixture_season)].copy()

        # Calculate features (same logic as used for the full fixtures prediction)
        home_team_historical_home = historical_subset[(historical_subset['team'] == home_team) & (historical_subset['venue'] == 'Home')].copy()
        home_goals_per_match = home_team_historical_home['gf'].mean() if not home_team_historical_home.empty else 0
        home_conceded_per_match = home_team_historical_home['ga'].mean() if not home_team_historical_home.empty else 0

        away_team_historical_away = historical_subset[(historical_subset['team'] == away_team) & (historical_subset['venue'] == 'Away')].copy()
        away_goals_per_match = away_team_historical_away['gf'].mean() if not away_team_historical_away.empty else 0
        away_conceded_per_match = away_team_historical_away['ga'].mean() if not away_team_historical_away.empty else 0

        home_team_historical = historical_subset[historical_subset['team'] == home_team].copy()
        home_wins = home_team_historical[home_team_historical['result'] == 'W'].shape[0]
        home_matches_played = home_team_historical.shape[0]
        home_win_rate = home_wins / home_matches_played if home_matches_played > 0 else 0

        away_team_historical = historical_subset[historical_subset['team'] == away_team].copy()
        away_wins = away_team_historical[away_team_historical['result'] == 'W'].shape[0]
        away_matches_played = away_team_historical.shape[0]
        away_win_rate = away_wins / away_matches_played if away_matches_played > 0 else 0

        home_team_recent_form = historical_subset[(historical_subset['team'] == home_team)].tail(5).copy()
        home_form_wins = (home_team_recent_form['result'] == 'W').sum()
        home_form_draws = (home_team_recent_form['result'] == 'D').sum()
        home_form_losses = (home_team_recent_form['result'] == 'L').sum()
        home_form_points = (home_team_recent_form['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))).sum()
        home_form_gf = home_team_recent_form['gf'].sum()
        home_form_ga = home_team_recent_form['ga'].sum()

        away_team_recent_form = historical_subset[(historical_subset['team'] == away_team)].tail(5).copy()
        away_form_wins = (away_team_recent_form['result'] == 'W').sum()
        away_form_draws = (away_team_recent_form['result'] == 'D').sum()
        away_form_losses = (away_team_recent_form['result'] == 'L').sum()
        away_form_points = (away_team_recent_form['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))).sum()
        away_form_gf = away_team_recent_form['gf'].sum()
        away_form_ga = away_team_recent_form['ga'].sum()

        h2h_matches = historical_subset[
            ((historical_subset['team'] == home_team) & (historical_subset['opponent'] == away_team)) |
            ((historical_subset['team'] == away_team) & (historical_subset['opponent'] == home_team))
        ].copy()

        h2h_home_wins = h2h_matches[(h2h_matches['team'] == home_team) & (h2h_matches['result'] == 'W')].shape[0] + \
                        h2h_matches[(h2h_matches['opponent'] == home_team) & (h2h_matches['result'] == 'L')].shape[0]
        h2h_draws = h2h_matches[h2h_matches['result'] == 'D'].shape[0]
        h2h_away_wins = h2h_matches[(h2h_matches['team'] == away_team) & (h2h_matches['result'] == 'W')].shape[0] + \
                        h2h_matches[(h2h_matches['opponent'] == away_team) & (h2h_matches['result'] == 'L')].shape[0]


        # Create a dictionary of features for the current fixture
        fixture_features = {
            'season': fixture_season,
            'home_goals_per_match': home_goals_per_match,
            'home_conceded_per_match': home_conceded_per_match,
            'away_goals_per_match': away_goals_per_match,
            'away_conceded_per_match': away_conceded_per_match,
            'home_win_rate': home_win_rate,
            'away_win_rate': away_win_rate,
            'home_form_wins': home_form_wins,
            'home_form_draws': home_form_draws,
            'home_form_losses': home_form_losses,
            'home_form_points': home_form_points,
            'home_form_gf': home_form_gf,
            'home_form_ga': home_form_ga,
            'away_form_wins': away_form_wins,
            'away_form_draws': away_form_draws,
            'away_form_losses': away_form_losses,
            'away_form_points': away_form_points,
            'away_form_gf': away_form_gf,
            'away_form_ga': away_form_ga,
            'h2h_home_wins': h2h_home_wins,
            'h2h_draws': h2h_draws,
            'h2h_away_wins': h2h_away_wins
        }
        specific_fixture_features_list.append(fixture_features)

    # Convert the list of dictionaries to a DataFrame
    specific_fixture_features_df = pd.DataFrame(specific_fixture_features_list)

    # Ensure the columns match the training data features
    specific_fixture_features_df = specific_fixture_features_df.reindex(columns=X_train.columns, fill_value=0)

    # Handle infinite values and fill NaNs in the created features
    specific_fixture_features_df = specific_fixture_features_df.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Use the best model to make predictions on the specific fixture features
    specific_fixture_predictions_encoded = best_model.predict(specific_fixture_features_df)

    # Convert encoded predictions back to labels
    specific_fixture_predictions = pd.Series(specific_fixture_predictions_encoded).map(reverse_target_mapping)

    # Add the predictions to the specific_fixtures dataframe
    specific_fixtures['predicted_result'] = specific_fixture_predictions.values

    # Display the specific_fixtures with the predicted results
    display(specific_fixtures[['date', 'home_team', 'away_team', 'predicted_result']])

    print("\n'W' means the Home Team is predicted to win.")
    print("'D' means a Draw is predicted.")
    print("'L' means the Home Team is predicted to lose (which means the Away Team is predicted to win).")

Enter the name of the Home Team: Real Madrid
Enter the name of the Away Team: Barcelona

Predicting matches between Real Madrid and Barcelona in the 2025-26 season:


,date,home_team,away_team,predicted_result
96,2025-10-26 20:00:00,Real Madrid,Barcelona,W
342,2026-05-10 19:00:00,Barcelona,Real Madrid,W



'W' means the Home Team is predicted to win.
'D' means a Draw is predicted.
'L' means the Home Team is predicted to lose (which means the Away Team is predicted to win).
